In [ ]:
!pip -q install langchain langchain_community pypdf fastembed chromadb

In [ ]:
# get the huggingface token
import os
from getpass import getpass
from google.colab import userdata

HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

Huggingface Token : ··········


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/Ahmed-MahmoudL-CVNIl.pdf")
data = loader.load()

In [ ]:
data[0].page_content

"Ahmed Mahmoud\nAhmedmamoud148@gmail.com\n \n01115312266\n \nNew Cairo\n \nLinkedIn-AM\n \nGitHub-AM\n \nEXPERIENCE\nDell Technologies (HackTrick Hackathon) - Data and Applied Science\n•Developed LSB steganography algorithm in Python for secure image-based message encoding, achieving 34% reduction in interception.\n•Engineered AI-driven message retrieval system with scoring algorithm (3 real, 6 fake chunks), improving data security by 39%.\nTECHNICAL PROJECTS\nAudio Insight (Ar& En)        Graduation Project, Docker, Git, Github, Aws, Ec2, S3, Whisper (fine-tuned), mT5 (fine-tuned), Flask, Prompt \nEn\n•Developed Audio Insight, an intelligent NLP web assistant that supports Arabic dialects (including Egyptian colloquial) and English, feature:\n•Audio Transcription: Fine-tuned a small version of OpenAI Whisper on Arabic and English speech using the fineweb_2 dataset.\n•Achieved a final WER of 7%, representing a 55% improvement over the pre-trained baseline.\n•Summarization: Fine-tuned G

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0
)

docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

47

In [ ]:
docs[0]

Document(metadata={'producer': 'Skia/PDF m135', 'creator': 'FlowCV - https://flowcv.com', 'creationdate': '2025-05-22T08:48:55+00:00', 'moddate': '2025-05-22T08:48:55+00:00', 'keywords': 'FlowCV – Online Resume Builder – https://flowcv.com', 'source': '/content/Ahmed-MahmoudL-CVNIl.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Ahmed Mahmoud\nAhmedmamoud148@gmail.com\n \n01115312266\n \nNew Cairo\n \nLinkedIn-AM\n \nGitHub-AM\n \nEXPERIENCE\nDell Technologies (HackTrick Hackathon) - Data and Applied Science')

In [ ]:
docs[1]

Document(metadata={'producer': 'Skia/PDF m135', 'creator': 'FlowCV - https://flowcv.com', 'creationdate': '2025-05-22T08:48:55+00:00', 'moddate': '2025-05-22T08:48:55+00:00', 'keywords': 'FlowCV – Online Resume Builder – https://flowcv.com', 'source': '/content/Ahmed-MahmoudL-CVNIl.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='•Developed LSB steganography algorithm in Python for secure image-based message encoding, achieving 34% reduction in interception.')

# Embding_model

In [ ]:
from langchain.embeddings.fastembed import FastEmbedEmbeddings


In [ ]:
model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large")

/usr/local/lib/python3.11/dist-packages/langchain_community/embeddings/fastembed.py:109: UserWarning: The model thenlper/gte-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  values["model"] = fastembed.TextEmbedding(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

# Vector_db

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
vectordb = Chroma.from_documents(
    docs,
    embedding_model,
    persist_directory="./chroma_db",

)

In [ ]:
Qa= "WWhat is name of unversity"

In [ ]:
vectordb.similarity_search(Qa)[0].page_content

'Audio Insight (Ar& En)        Graduation Project, Docker, Git, Github, Aws, Ec2, S3, Whisper (fine-tuned), mT5 (fine-tuned), Flask, Prompt \nEn'

In [ ]:
# initialize the retriever
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 3})

In [ ]:
retriever.invoke("WWhat is name of unversity")[0].page_content

'Audio Insight (Ar& En)        Graduation Project, Docker, Git, Github, Aws, Ec2, S3, Whisper (fine-tuned), mT5 (fine-tuned), Flask, Prompt \nEn'

## it's very bad and i except tis becuse bad embding model

In [ ]:
!pip install -q  langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
# import the question-answering chain and Huggingface Hub LLM
from langchain_huggingface import HuggingFaceEndpoint

# define the llm
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    temperature=0.1,
    max_new_tokens=512,
    return_full_text=False,
    repetition_penalty=1.1,
    top_p=0.9
)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain = (
{"context": retriever, "query": RunnablePassthrough()}
| prompt
| llm
| output_parser
)
